In [1]:
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from xgboostextension import XGBRanker 
from xgboost import XGBModel
import pandas as pd
from sklearn.metrics import confusion_matrix

In [2]:
cd

/Users/viktorvandervalk


In [3]:
DTYPES = {

    'srch_id'                    : np.int32,
    'visitor_location_country_id': np.int32,
    'visitor_hist_starrating'    : np.float32,
    'visitor_hist_adr_usd'       : np.float32,
    'prop_country_id'            : np.int32,
    'prop_id'                    : np.int32,
    'prop_starrating'            : np.float32,
    'prop_review_score'          : np.float32,
    'prop_brand_bool'            : np.bool,
    'prop_location_score1'       : np.float32,
    'prop_location_score2'       : np.float32,
    'prop_log_historical_price'  : np.float32,
    'position'                   : np.int32,
    'price_usd'                  : np.float32,
    'promotion_flag'             : np.bool,
    'srch_destination_id'        : np.int32,
    'srch_length_of_stay'        : np.float32,
    'srch_booking_window'        : np.float32,
    'srch_adults_count'          : np.float32,
    'srch_children_count'        : np.float32,
    'srch_room_count'            : np.float32,
    'srch_saturday_night_bool'   : np.bool,
    'srch_query_affinity_score'  : np.float32,
    'random_bool'                : np.bool,
    'click_bool'                 : np.bool,
    'booking_bool'               : np.bool,
    'competitors'                : np.float32,
    'prop_reviewed'              : np.bool,
    'avg_orig_dest_dist'         : np.float32,
    'std_orig_dest_dist'         : np.float32,
    'domestic_travel'            : np.bool,
}

FILE = './Desktop/Vakken/DataMiningTechniques/training_final.csv'

data = pd.read_csv(FILE, index_col=0, dtype=DTYPES)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
data.head()

Unnamed: 0.1  srch_id  visitor_location_country_id  \
0             0        1                          187   
1             1        1                          187   
2             2        1                          187   
3             3        1                          187   
4             4        1                          187   

   visitor_hist_starrating  visitor_hist_adr_usd  prop_starrating  \
0                      NaN                   NaN        -0.171762   
1                      NaN                   NaN         0.779692   
2                      NaN                   NaN        -0.171762   
3                      NaN                   NaN        -1.123215   
4                      NaN                   NaN         0.779692   

   prop_review_score  prop_brand_bool  prop_location_score1  \
0          -0.256862             True             -0.027817   
1           0.215022             True             -0.439310   
2           0.686906             True             -0.439310   
3           0.215022             True             -0.027817   
4          -0.256862             True             -0.151918   

   prop_location_score2       ...        srch_query_affinity_score  \
0             -0.614763       ...                              0.0   
1             -0.819956       ...                              0.0   
2             -0.751795       ...                              0.0   
3             -0.836996       ...                              0.0   
4             -0.044625       ...                              0.0   

   random_bool  click_bool  booking_bool  competitors  prop_reviewed  \
0         True       False         False          0.0           True   
1         True       False         False          0.0           True   
2         True       False         False          0.0           True   
3         True       False         False         -1.0           True   
4         True       False         False          0.0           True   

   avg_orig_dest_dist  std_orig_dest_dist  domestic_travel  global_prop_id  
0           -0.120946           -0.481842            False         893_219  
1           -0.120946           -0.481842            False       10404_219  
2           -0.120946           -0.481842            False       21315_219  
3           -0.120946           -0.481842            False       27348_219  
4           -0.120946           -0.481842            False       29604_219  

[5 rows x 31 columns]

In [13]:
TestSize = 300000
FilterStep = 7
data_try_train = data.iloc[:TestSize]
data_train_filtered = data_try_train
data_train_filtered.loc[:,'relevance'] = 4*data_train_filtered.loc[:,'booking_bool'] + 1*data_train_filtered.loc[:,'click_bool']
data_train_filtered.loc[:,"ideal_rank"] = data_train_filtered.groupby("srch_id")["relevance"].rank(method = 'first',ascending=False)
data_train_cut = data_train_filtered[data_train_filtered['prop_reviewed'] == False]
data_train_filtered = data_train_filtered[data_train_filtered['ideal_rank'] < FilterStep]
data_train_cut = data_train_cut[data_train_cut['ideal_rank'] < FilterStep]


X_train = data_try_train[['prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1','prop_location_score2', 'prop_log_historical_price',
       'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score','random_bool', 'competitors', 'avg_orig_dest_dist',
       'std_orig_dest_dist', 'domestic_travel']]

X_train_filtered = data_train_filtered[['prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1','prop_location_score2', 'prop_log_historical_price',
       'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score','random_bool', 'competitors', 'avg_orig_dest_dist',
       'std_orig_dest_dist', 'domestic_travel']]


Y_data_train = data_try_train[['srch_id','click_bool','booking_bool']]
Y_data_train_filtered = data_train_filtered[['srch_id','click_bool','booking_bool']]



data_try_test = data.iloc[TestSize:2*TestSize]

X_test = data_try_test[['prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1','prop_location_score2', 'prop_log_historical_price',
       'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score','random_bool', 'competitors', 'avg_orig_dest_dist',
       'std_orig_dest_dist', 'domestic_travel']]

Y_data_test = data_try_test[['srch_id','click_bool','booking_bool']]

Y_data_test.loc[:,'relevance'] = 4*Y_data_test.loc[:,'booking_bool'] + 1*Y_data_test.loc[:,'click_bool']
Y_data_test.loc[:,"ideal_rank"] = Y_data_test.groupby("srch_id")["relevance"].rank(method = 'first',ascending=False)
Y_data_test.loc[:,'ideal_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['ideal_rank']+1)




/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
from sklearn import linear_model

tuning = []
clf_clicked = linear_model.SGDClassifier(loss = 'log', penalty='elasticnet', max_iter=1000, learning_rate='optimal', l1_ratio = 0.5)
clf_booked = linear_model.SGDClassifier(loss = 'log', penalty='elasticnet', max_iter=1000, learning_rate='optimal', l1_ratio = 0.5)


# Fit the classifier to the training set
clf_clicked.fit(X_train_filtered.values, Y_data_train_filtered['click_bool'].values)
clf_booked.fit(X_train_filtered.values, Y_data_train_filtered['booking_bool'].values)

Y_data_test.loc[:,'click_predict'] = clf_clicked.predict(X_test.values)
Y_data_test.loc[:,'clicked_proba'] = clf_clicked.predict_proba(X_test.values)[:,1]

Y_data_test.loc[:,'book_predict'] = clf_booked.predict(X_test.values)
Y_data_test.loc[:,'booked_proba'] = clf_booked.predict_proba(X_test.values)[:,1]

Y_data_test.loc[:,'combi_proba'] = Y_data_test.loc[:,'booked_proba'] + Y_data_test.loc[:,'clicked_proba']


Y_data_test.loc[:,"booked_rank"] = Y_data_test.groupby("srch_id")["booked_proba"].rank(method = 'first',ascending=False)
Y_data_test.loc[:,"clicked_rank"] = Y_data_test.groupby("srch_id")["clicked_proba"].rank(method = 'first',ascending=False)
Y_data_test.loc[:,"combi_rank"] = Y_data_test.groupby("srch_id")["combi_proba"].rank(method = 'first',ascending=False)

Y_data_test.loc[:,'booked_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['booked_rank']+1)
Y_data_test.loc[:,'clicked_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['clicked_rank']+1)
Y_data_test.loc[:,'combi_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['combi_rank']+1)


con_clicked = confusion_matrix(Y_data_test['click_bool'], Y_data_test['click_predict'])
TruePos_clicked = con_clicked[1,1]/con_clicked[1].sum()

con_booked = confusion_matrix(Y_data_test['booking_bool'], Y_data_test['book_predict'])
TruePos_booked = con_booked[1,1]/con_booked[1].sum()

booked_nDCGp = Y_data_test['booked_gain'].sum()/Y_data_test['ideal_gain'].sum()
clicked_nDCGp = Y_data_test['clicked_gain'].sum()/Y_data_test['ideal_gain'].sum()
combi_nDCGp = Y_data_test['combi_gain'].sum()/Y_data_test['ideal_gain'].sum()
print(booked_nDCGp, clicked_nDCGp, combi_nDCGp)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.43908146487825417 0.44213646592376843 0.44118550659242356


In [18]:
import sklearn
sklearn.__version__

'0.19.1'

In [23]:
print(booked_nDCGp, clicked_nDCGp, combi_nDCGp)

0.4358776028837137

In [16]:
print(booked_nDCGp, clicked_nDCGp, combi_nDCGp)

0.4437269647003759 0.4164429908905808 0.44108399056709086


In [7]:
tuning = []
for n_estimators in [200]:
    for eta in [0.1]:
        for min_child_weight in [1,2]:
            for max_depth in [3]:
                for gamma in [1,2]:
                    print('-------NewRound------')
                    xg_cl_clicked = xgb.XGBClassifier(n_estimators=n_estimators,eta=eta, min_child_weight=min_child_weight, max_depth=max_depth, gamma=gamma, subsample=0.1, objective='rank:pairwise', eval_metric = "logloss",seed=123)
                    xg_cl_booked = xgb.XGBClassifier(n_estimators=n_estimators,eta=eta, min_child_weight=min_child_weight, max_depth=max_depth, gamma=gamma, subsample=0.1, objective='rank:pairwise',eval_metric = "logloss", seed=123)

                    
                    # Fit the classifier to the training set
                    xg_cl_clicked.fit(X_train_filtered.values, Y_data_train_filtered['click_bool'].values)
                    xg_cl_booked.fit(X_train_filtered.values, Y_data_train_filtered['booking_bool'].values)

                    Y_data_test.loc[:,'click_predict'] = xg_cl_clicked.predict(X_test.values)
                    Y_data_test.loc[:,'click_proba'] = xg_cl_clicked.predict_proba(X_test.values)[:,1]

                    Y_data_test.loc[:,'book_predict'] = xg_cl_booked.predict(X_test.values)
                    Y_data_test.loc[:,'booked_proba'] = xg_cl_booked.predict_proba(X_test.values)[:,1]
                    
                    Y_data_test.loc[:,'combi_proba'] = Y_data_test.loc[:,'booked_proba'] * Y_data_test.loc[:,'click_proba'] 
                    Y_data_test.loc[:,'combi2_proba'] = Y_data_test.loc[:,'booked_proba'] + Y_data_test.loc[:,'click_proba'] 
                    Y_data_test.loc[:, "random_score"] = np.random.random(len(Y_data_test["combi_proba"]))

                    Y_data_test.loc[:,"click_rank"] = Y_data_test.groupby("srch_id")["booked_proba"].rank(method = 'first',ascending=False)
                    Y_data_test.loc[:,"book_rank"] = Y_data_test.groupby("srch_id")["click_proba"].rank(method = 'first',ascending=False)
                    Y_data_test.loc[:,"combi_rank"] = Y_data_test.groupby("srch_id")["combi_proba"].rank(method = 'first',ascending=False)
                    Y_data_test.loc[:,"combi2_rank"] = Y_data_test.groupby("srch_id")["combi2_proba"].rank(method = 'first',ascending=False)
                    Y_data_test.loc[:, "random_rank"] = Y_data_test.groupby("srch_id")["random_score"].rank(method = 'first',ascending=False)

                    Y_data_test.loc[:,'click_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['click_rank']+1)
                    Y_data_test.loc[:,'book_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['book_rank']+1)
                    Y_data_test.loc[:,'combi_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['combi_rank']+1)
                    Y_data_test.loc[:,'combi2_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['combi2_rank']+1)
                    Y_data_test.loc[:,'random_gain'] = Y_data_test['relevance']/np.log2(Y_data_test['random_rank']+1)
                

                    con_clicked = confusion_matrix(Y_data_test['click_bool'], Y_data_test['click_predict'])
                    TruePos_clicked = con_clicked[1,1]/con_clicked[1].sum()

                    con_booked = confusion_matrix(Y_data_test['booking_bool'], Y_data_test['book_predict'])
                    TruePos_booked = con_booked[1,1]/con_booked[1].sum()
                    
                    click_nDCGp = Y_data_test['click_gain'].sum()/Y_data_test['ideal_gain'].sum()
                    book_nDCGp = Y_data_test['book_gain'].sum()/Y_data_test['ideal_gain'].sum()
                    combi_nDCGp = Y_data_test['combi_gain'].sum()/Y_data_test['ideal_gain'].sum()
                    combi2_nDCGp = Y_data_test['combi2_gain'].sum()/Y_data_test['ideal_gain'].sum()
                    random_nDCGp = Y_data_test['random_gain'].sum()/Y_data_test['ideal_gain'].sum()
                    
                    tuning.append([combi_nDCGp, combi2_nDCGp, random_nDCGp, click_nDCGp, book_nDCGp, TruePos_clicked, TruePos_booked, n_estimators, eta, min_child_weight, max_depth, gamma])
scores = pd.DataFrame(tuning, columns = ['combi_nDCGp','combi2_nDCGp','random_nDCGp','click_nDCGp','book_nDCGp','TruePos_clicked', 'TruePos_booked', 'n_estimators', 'eta', 'min_child_weight', 'max_depth', 'gamma'])
print(scores)
#scores.to_csv('Hyperparameter Tuning')

-------NewRound------


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

-------NewRound------


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is

-------NewRound------


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is

-------NewRound------


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is

   combi_nDCGp  combi2_nDCGp  random_nDCGp  click_nDCGp  book_nDCGp  \
0     0.296383      0.485793      0.353902     0.468230    0.449316   
1     0.291538      0.483489      0.357338     0.447214    0.475438   
2     0.296204      0.474693      0.367051     0.449540    0.458016   
3     0.287304      0.467273      0.338446     0.442766    0.442241   

   TruePos_clicked  TruePos_booked  n_estimators  eta  min_child_weight  \
0              0.0             0.0           200  0.1                 1   
1              0.0             0.0           200  0.1                 1   
2              0.0             0.0           200  0.1                 2   
3              0.0             0.0           200  0.1                 2   

   max_depth  gamma  
0          3      1  
1          3      2  
2          3      1  
3          3      2  


In [113]:
con_clicked = confusion_matrix(Y_data_test['click_bool'], Y_data_test['click_predict'])

In [136]:
con_booked[1,1]

6056

In [93]:
Y_data_test['gain'].sum()

172.00093514461872

In [10]:
con2_booked

array([[2911,    2],
       [  85,    2]])

In [9]:
con_booked

array([[2793,  120],
       [  79,    8]])

In [205]:
Y_booked_rank

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [220]:
con_booked

array([[930,  40],
       [ 29,   1]])

In [179]:
Y_clicked_test.value_counts()

False    284
True      16
Name: click_bool, dtype: int64

In [162]:
a = []
for x in range(5):
    for y in range(10):
         a.append([x,y])
wa = pd.DataFrame(a, columns = ['x','y'])

In [163]:
wa

x  y
0   0  0
1   0  1
2   0  2
3   0  3
4   0  4
5   0  5
6   0  6
7   0  7
8   0  8
9   0  9
10  1  0
11  1  1
12  1  2
13  1  3
14  1  4
15  1  5
16  1  6
17  1  7
18  1  8
19  1  9
20  2  0
21  2  1
22  2  2
23  2  3
24  2  4
25  2  5
26  2  6
27  2  7
28  2  8
29  2  9
30  3  0
31  3  1
32  3  2
33  3  3
34  3  4
35  3  5
36  3  6
37  3  7
38  3  8
39  3  9
40  4  0
41  4  1
42  4  2
43  4  3
44  4  4
45  4  5
46  4  6
47  4  7
48  4  8
49  4  9

In [137]:
np.argmax(Y2_proba[subgroup,1])


15

In [138]:
subgroup

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
           dtype='int64')

In [134]:
subgroup[3]

31

In [148]:
Y2_proba[subgroup,1]

array([ 1.1912895 ,  1.0451188 ,  0.8921292 , -1.7861435 ,  0.9353764 ,
        0.9702943 ,  0.72785723,  0.67367446,  1.0884798 ,  0.946281  ,
        0.8812214 ,  0.4744297 ,  0.1845507 ,  0.09598336,  0.3475676 ,
        0.14491755,  0.8288474 ,  0.29219383,  0.24206164,  0.08326882,
        0.11208385,  0.58317137,  0.07352605,  0.23112434,  0.24206164],
      dtype=float32)

In [140]:
y_ranked_booked

array([0., 0., 0., ..., 0., 0., 0.])

In [147]:
confusion_matrix(Y2_test, y_ranked_booked)

array([[9351,  383],
       [ 237,   29]])

In [96]:
Y2_test.value_counts()

False    9734
True      266
Name: booking_bool, dtype: int64

In [110]:
Y2_pred[10:20]

array([False, False, False, False, False,  True, False, False, False,
       False])

In [113]:
Y2_proba[:,0]

array([4.1821356, 1.8880485, 2.3451734, ..., 3.9283028, 5.3698034,
       3.3354583], dtype=float32)

In [ ]:
gbm_param_grid = {
    'n_estimators': [25, 50, 100],
    'eta' : [0.001, 0.01, 0.1]
    'min_child_weight' : np.arange(1,5)
    'max_depth': range(4, 12)
    'gamma':np.arange(0,5,2)
    'subsample':[0.01]
    'objective':"binary:logistic"
}

# Instantiate the regressor: gbm
gbm = xgb.XGBClassifier()

# Perform random search: grid_mse
#randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid, scoring='neg_mean_squared_error', n_iter=100, cv=10, verbose=1)
#randomized_mse.fit(X,y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

#xg_cl = xgb.train(dtrain=dtrain, params=params, num_boost_round=20)

#Predict the labels of the test set: preds
preds = xg_ranker.predict(X_test.values)

In [199]:
X_train = data.iloc[:-5,:4]
X_test = data.iloc[-5:,:4]

Y_train = data.iloc[:-5,4]
Y_test = data.iloc[-5:,4]

dtrain = xgb.DMatrix(data=X_train,label=Y_train)
dtest = xgb.DMatrix(data=X_test, label=Y_test)

#dtrain.set_group([4,8])

xg_ranker = XGBRanker()
xg_cl = xgb.XGBClassifier(n_estimators=100, objective='rank:pairwise', seed=124)

# Fit the classifier to the training set
xg_ranker.fit(X_train.values, Y_train.values)

xg_cl.fit(X_train,Y_train)

#params={'eta':0.1,'max_depth':3, 'learning_rate':0.1, 'n_estimators':100, 'objective':"binary:logistic"}


gbm_param_grid = {
    'n_estimators': [25, 50, 100],
    'eta' : [0.001, 0.01, 0.1]
    'min_child_weight' : np.arange(1,5)
    'max_depth': range(4, 12)
    'gamma':np.arange(0,5,2)
    'subsample':[0.01]
    'objective':"binary:logistic"
}

# Instantiate the regressor: gbm
gbm = xgb.XGBClassifier()

# Perform random search: grid_mse
randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid, scoring='neg_mean_squared_error', n_iter=100, cv=10, verbose=1)
randomized_mse.fit(X,y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

#xg_cl = xgb.train(dtrain=dtrain, params=params, num_boost_round=20)

#Predict the labels of the test set: preds
preds = xg_ranker.predict(X_test.values)

preds2 = xg_cl.predict(X_test)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
np.arange(0,5,2)

array([0, 2, 4])

In [201]:
proba = xg_cl.predict_proba(X_test)

In [203]:
proba[:,2]

array([0.99652123, 0.98708886, 0.9970553 , 0.99361295, 0.9528438 ],
      dtype=float32)

In [184]:
Y_test

145    2
146    2
147    2
148    2
149    2
Name: Label, dtype: int64

In [35]:
pred3

array([[-1.0492768,  2.0492768],
       [-1.0492768,  2.0492768],
       [ 2.0492768, -1.0492767]], dtype=float32)